# Build a ChatBot

In [1]:
import os

os.chdir("../../../")

In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

## Loading the Model

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1b26c91f-c81e-4ed8-9618-eba323da3ba1-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})

In [5]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I do not have access to your personal information, so I do not know your name.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 12, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-954bdc94-915f-4454-b330-2b19e160f07e-0', usage_metadata={'input_tokens': 12, 'output_tokens': 22, 'total_tokens': 34})

The model does not have access to previous information we provided. To overcome that, we can simply give the whole context to it:

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2f6055f1-e960-472e-876d-c5217b6ae4d6-0', usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40})

## Message History

We can use a Message History class to wrap our model and make it stateful.

In [7]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [8]:
config = {"configurable": {"session_id": "abc2"}}

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config
)

response.content

'Hello Bob! How can I assist you today?'

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

response.content

'Your name is Bob.'

In [14]:
# if we change the config to another session_id, it will start afresh
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, I don't have access to that information."

In [15]:
# However, we can always go back to the original conversation 
# (since we are persisting it in a database)

config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob.'

## Prompt Templates

Let's create a prompt template and wrap it around the Messages History:

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [17]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm Bob")]})

response.content

'Hello Bob! How can I assist you today?'

In [18]:
# We can now wrap this in the same Messages History object as before
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [19]:
config = {"configurable": {"session_id": "abc5"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

'Hello, Jim! How can I help you today?'

In [20]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim.'

Let's make the prompt a bit more complex:

In [21]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [22]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'

In [23]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [24]:
config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola Todd! ¿En qué puedo ayudarte hoy?'

In [25]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content

'Tu nombre es Todd.'

## Managing Conversation History

We have to manage the size of the conversation history to avoid it growing unbounded and overflowing the context window of the LLM. 

Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.

In [26]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

Now if we try asking the model our name, it won't know it since we trimmed that part of the chat history:

In [27]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)

response.content

"I'm sorry, I don't have access to personal information like your name. How can I assist you today?"

In [28]:
# it still remembers the last tokens
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    }
)

response.content

'You asked "what\'s 2 + 2?"'

In [29]:
# Let's now wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, I don't have access to your personal information, including your name."

## Streaming

In [30]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

|Sure|,| Todd|!| Why| couldn|'t| the| bicycle| stand| up| by| itself|?| Because| it| was| two|-t|ired|!| 😄||